Fuzzy Association Rule Mining

### 🧩 Fuzzy Association Rule Mining (FARM)

In this section, we use **Fuzzy Logic** and **Association Rule Mining** to extract interpretable rules from the anemia dataset.

- 📊 **Step 1–2**: Load the dataset and normalize numerical features to the range [0, 1].
- 🌫️ **Step 3**: Define fuzzy membership functions (Low, Medium, High) for each feature using triangular membership functions via `scikit-fuzzy`.
- 🧠 **Step 4**: Assign each feature a fuzzy category based on maximum membership.
- 📦 **Step 5**: Convert the fuzzy-labeled dataset to a one-hot encoded format for rule mining.

- 🧮 **Step 6**: Apply the **Apriori algorithm** to find frequent fuzzy itemsets using `mlxtend`.
- 🔗 **Step 7**: Generate fuzzy association rules with support and confidence metrics.
- 📃 **Step 8–10**: Display rules in human-readable format and save outputs as `.csv` and `.txt`.

#### ✅ Sample Output:
```plaintext
IF {Result_Anemia, MCHC_Medium} THEN {Hemoglobin_Medium}: Confidence = 0.93
IF {Hemoglobin_Medium, MCHC_Medium} THEN {Result_Anemia}: Confidence = 0.70



These rules can help clinicians interpret diagnostic patterns using fuzzy linguistic terms — enabling transparent decision support.

💾 All fuzzy rules are also saved as:

- fuzzy_association_rules.csv

- fuzzy_output.txt

In [1]:
import numpy as np
import pandas as pd
import skfuzzy as fuzz
from mlxtend.frequent_patterns import apriori, association_rules

# 1. Load dataset
file_path = r"C:\Users\Andromeda\Downloads\datasets\anemia_cleaned_70rows.csv"
df = pd.read_csv(file_path)

# 2. Normalize Features
X = df.iloc[:, :-1]  # First 4 columns (features)
y = df.iloc[:, -1]   # Last column (target)

X_min = X.min()
X_max = X.max()
X_norm = (X - X_min) / (X_max - X_min)

# 3. Define fuzzy membership functions
def fuzzify_feature(data, low_range, medium_range, high_range):
    """Generate fuzzy membership values for a feature."""
    low = fuzz.trimf(data, low_range)
    medium = fuzz.trimf(data, medium_range)
    high = fuzz.trimf(data, high_range)
    return np.vstack((low, medium, high)).T

# Apply fuzzy sets for each feature
hemoglobin_fuzzy = fuzzify_feature(X_norm['Hemoglobin'], [0, 0, 0.5], [0.2, 0.5, 0.8], [0.5, 1, 1])
mch_fuzzy = fuzzify_feature(X_norm['MCH'], [0, 0, 0.5], [0.2, 0.5, 0.8], [0.5, 1, 1])
mchc_fuzzy = fuzzify_feature(X_norm['MCHC'], [0, 0, 0.5], [0.2, 0.5, 0.8], [0.5, 1, 1])
mcv_fuzzy = fuzzify_feature(X_norm['MCV'], [0, 0, 0.5], [0.2, 0.5, 0.8], [0.5, 1, 1])

# 4. Convert to categorical fuzzy labels
fuzzy_labels = ['Low', 'Medium', 'High']
df_fuzzy = pd.DataFrame({
    'Hemoglobin': [fuzzy_labels[np.argmax(row)] for row in hemoglobin_fuzzy],
    'MCH': [fuzzy_labels[np.argmax(row)] for row in mch_fuzzy],
    'MCHC': [fuzzy_labels[np.argmax(row)] for row in mchc_fuzzy],
    'MCV': [fuzzy_labels[np.argmax(row)] for row in mcv_fuzzy],
    'Result': y.map({0: 'No Anemia', 1: 'Anemia'})
})

# 5. Convert fuzzy dataset to transactional format
df_encoded = pd.get_dummies(df_fuzzy)

# 6. Apply Apriori Algorithm
frequent_itemsets = apriori(df_encoded, min_support=0.2, use_colnames=True)

# 7. Generate association rules
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

# 8. Display extracted fuzzy association rules
print("Extracted Fuzzy Association Rules:")
print(rules[['antecedents', 'consequents', 'support', 'confidence']])

# 9. Save extracted rules
rules.to_csv("fuzzy_association_rules.csv", index=False)

# 10. Print and Save Frequent Itemsets and Rules
def print_frequent_itemsets(frequent_items):
    """Print frequent fuzzy itemsets."""
    print("\nFrequent Fuzzy Itemsets:")
    for index, row in frequent_items.iterrows():
        print(f"{set(row['itemsets'])}: Support = {row['support']:.2f}")

def print_fuzzy_rules(rules):
    """Print fuzzy association rules in a readable format."""
    print("\nFuzzy Association Rules:")
    for index, row in rules.iterrows():
        antecedents = ", ".join(list(row["antecedents"]))
        consequents = ", ".join(list(row["consequents"]))
        print(f"IF {{{antecedents}}} THEN {{{consequents}}}: Confidence = {row['confidence']:.2f}")

def save_output_to_file(frequent_items, rules, filename="fuzzy_output.txt"):
    """Save frequent itemsets and rules to a text file."""
    with open(filename, "w", encoding="utf-8") as f:
        f.write("Frequent Fuzzy Itemsets:\n")
        for index, row in frequent_items.iterrows():
            f.write(f"{set(row['itemsets'])}: Support = {row['support']:.2f}\n")

        f.write("\nFuzzy Association Rules:\n")
        for index, row in rules.iterrows():
            antecedents = ", ".join(row["antecedents"])
            consequents = ", ".join(row["consequents"])
            f.write(f"IF {{{antecedents}}} THEN {{{consequents}}}: Confidence = {row['confidence']:.2f}\n")

# Print results
print_frequent_itemsets(frequent_itemsets)
print_fuzzy_rules(rules)

# Save results to a file
save_output_to_file(frequent_itemsets, rules)


Extracted Fuzzy Association Rules:
                           antecedents                          consequents  \
0                         (MCH_Medium)                    (Hemoglobin_High)   
1                   (Result_No Anemia)                    (Hemoglobin_High)   
2                    (Hemoglobin_High)                   (Result_No Anemia)   
3                  (Hemoglobin_Medium)                        (MCHC_Medium)   
4                        (MCHC_Medium)                  (Hemoglobin_Medium)   
5                            (MCV_Low)                  (Hemoglobin_Medium)   
6                  (Hemoglobin_Medium)                      (Result_Anemia)   
7                      (Result_Anemia)                  (Hemoglobin_Medium)   
8                         (MCH_Medium)                   (Result_No Anemia)   
9                      (Result_Anemia)                        (MCHC_Medium)   
10                       (MCHC_Medium)                   (Result_No Anemia)   
11               